In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np

# Load the MoveNet Thunder model from TensorFlow Hub
model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")

# Define the pairs of keypoints that make up the skeleton
KEYPOINT_EDGE_PAIRS = [
    (0, 1), (0, 2), (1, 3), (2, 4), (0, 5), (0, 6), (5, 7), (7, 9), (6, 8), (8, 10), 
    (5, 6), (5, 11), (6, 12), (11, 12), (11, 13), (13, 15), (12, 14), (14, 16)
]

# Function to run inference
def movenet(input_image):
    movenet = model.signatures['serving_default']
    
    # Prepare the input image
    input_image = tf.image.resize_with_pad(input_image, 256, 256)
    input_image = tf.cast(input_image, dtype=tf.int32)
    input_image = tf.expand_dims(input_image, axis=0)
    
    # Run inference
    outputs = movenet(input_image)
    keypoints = outputs['output_0']
    
    return keypoints

# Function to draw keypoints and skeleton on the image
def draw_keypoints_and_skeleton(image, keypoints, threshold=0.11):
    y, x, c = image.shape
    shaped = np.squeeze(np.multiply(keypoints, [y, x, 1]))

    # Draw keypoints
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > threshold:
            cv2.circle(image, (int(kx), int(ky)), 4, (0, 255, 0), -1)

    # Draw skeleton
    for edge_pair in KEYPOINT_EDGE_PAIRS:
        pt1 = shaped[edge_pair[0]]
        pt2 = shaped[edge_pair[1]]
        if pt1[2] > threshold and pt2[2] > threshold:
            cv2.line(image, (int(pt1[1]), int(pt1[0])), (int(pt2[1]), int(pt2[0])), (0, 255, 0), 2)

    return image

# Initialize the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Run pose estimation
    keypoints = movenet(tf.convert_to_tensor(image_rgb))
    
    # Draw keypoints and skeleton on the frame
    frame_with_keypoints = draw_keypoints_and_skeleton(frame, keypoints.numpy())
    
    # Display the frame with keypoints and skeleton
    cv2.imshow('Human Pose Estimation', frame_with_keypoints)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
